## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```

In [1]:
from fastembed import TextEmbedding
import numpy as np

e:\LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* **-0.11**
* 0
* 0.51

In [3]:
model_name = 'jinaai/jina-embeddings-v2-small-en'
embedder = TextEmbedding(model_name=model_name)

query = 'I just discovered the course. Can I join now?'
query_embedding = next(embedder.embed([query]))
print("Q1 - Minimal value:", np.min(query_embedding))

Q1 - Minimal value: -0.11726373885183883


## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* **0.9**

In [4]:
doc_text = 'Can I still join the course after the start date?'
doc_embedding = next(embedder.embed([doc_text]))
cos_sim = np.dot(query_embedding, doc_embedding)
print("Q2 - Cosine similarity:", cos_sim)

Q2 - Cosine similarity: 0.9008528895674548


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- **1**
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.


In [5]:
documents = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
     'question': 'Course - Can I still join the course after the start date?'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
     'question': 'Course - Can I follow the course after it finishes?'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00...",
     'question': 'Course - When will the course start?'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3...',
     'question': 'Course - What can I do before the course starts?'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
     'question': 'How can we contribute to the course?'}
]

text_embeddings = list(embedder.embed([doc['text'] for doc in documents]))
text_matrix = np.stack(text_embeddings)
sims_text_only = text_matrix @ query_embedding
print("Q3 - The document index with the highest similarity:", int(np.argmax(sims_text_only)))

Q3 - The document index with the highest similarity: 1


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- **0**
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [6]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_embeddings = list(embedder.embed(full_texts))
full_matrix = np.stack(full_embeddings)
sims_full = full_matrix @ query_embedding
print("Q4 - The document index with the highest similarity:", int(np.argmax(sims_full)))

Q4 - The document index with the highest similarity: 0


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- **384**
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [ ]:
from fastembed import list_supported_models

models = list_supported_models()
for model in models:
    print(model['name'], "- Dimensiones:", model['size'])

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- **0.87**
- 0.77
- 0.67

Run Qdrant in Docker:
```bash
docker run -p 6333:6333 qdrant/qdrant
```

In [ ]:
from fastembed.embedding import TextEmbedding
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import requests

embedder = TextEmbedding(model_name="BAAI/bge-small-en")

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = 'machine-learning-zoomcamp'
        documents.append(doc)

texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

vectors = []
batch_size = 16  # Ajusta esto según tu RAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    batch_vectors = list(embedder.embed(batch))
    vectors.extend(batch_vectors)

client = QdrantClient(host="localhost", port=6333)

client.recreate_collection(
    collection_name="ml_faq",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

points = [
    PointStruct(id=i, vector=vectors[i], payload={"text": texts[i]})
    for i in range(len(vectors))
]
client.upsert(collection_name="ml_faq", points=points)

query = 'I just discovered the course. Can I join now?'
query_vector = next(embedder.embed([query]))

results = client.search(
    collection_name="ml_faq",
    query_vector=query_vector,
    limit=1
)

print("Q6 - Score más alto:", results[0].score)


C:\Users\sanch\AppData\Local\Temp\ipykernel_17704\689795090.py:36: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Q6 - Score más alto: 0.8703172


C:\Users\sanch\AppData\Local\Temp\ipykernel_17704\689795090.py:53: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
